In [1]:
!pip install -q xgboost shap


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


In [3]:
DATA_PATH = "/content/drive/MyDrive/default_credit.csv"
TARGET_COL = "Y"     # correct label column
OUTPUT_DIR = "/content/drive/MyDrive/project_outputs"
RANDOM_STATE = 42
TEST_SIZE = 0.2

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Load data
df = pd.read_csv(DATA_PATH)

# Drop index-like column
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

print("Loaded data with shape:", df.shape)
print("Columns:", df.columns.tolist())

# 2. Basic cleaning
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove target from feature lists
if TARGET_COL in num_cols:
    num_cols.remove(TARGET_COL)
if TARGET_COL in cat_cols:
    cat_cols.remove(TARGET_COL)

# Handle missing
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df[cat_cols] = df[cat_cols].fillna("MISSING")
# --- Safe cleaning for the target column Y ---
# (paste this where you were getting the ValueError)

# 1) Inspect a few distinct values in Y to see what's wrong
print("Unique (sample) values in Y (first 20):", pd.Series(df['Y'].unique()).head(20).tolist())

# 2) Try coercing to numeric and see how many non-convertible values there are
y_numeric = pd.to_numeric(df['Y'], errors='coerce')
n_non_numeric = y_numeric.isna().sum()
print("Non-convertible values in Y:", n_non_numeric)

if n_non_numeric > 0:
    # Show examples of non-numeric entries (so you can inspect)
    bad_values = df.loc[y_numeric.isna(), 'Y'].unique().tolist()
    print("Examples of bad values in Y:", bad_values)
    # Common case: the very first row is an extra header like 'default payment next month'
    # If so, drop that row(s). We'll drop any rows where Y cannot be converted to numeric.
    df = df.loc[~y_numeric.isna()].copy()
    # Recompute numeric column after dropping
    df['Y'] = pd.to_numeric(df['Y'], errors='coerce')
else:
    # All good — assign numeric
    df['Y'] = y_numeric

# 3) Final cast to int (safe now because non-numeric rows were dropped)
df['Y'] = df['Y'].astype(int)

# 4) Quick sanity checks
print("After cleaning, shape:", df.shape)
print("Target value counts:\n", df['Y'].value_counts())

# If you dropped rows and want to know how many:
# original_count = 30001  # or store before cleaning
# print("Dropped rows:", original_count - df.shape[0])

# 3. Split
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
)

print("Train/test sizes:", X_train.shape, X_test.shape)


Loaded data with shape: (30001, 24)
Columns: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'Y']
Unique (sample) values in Y (first 20): ['default payment next month', '1', '0']
Non-convertible values in Y: 1
Examples of bad values in Y: ['default payment next month']
After cleaning, shape: (30000, 24)
Target value counts:
 Y
0    23364
1     6636
Name: count, dtype: int64
Train/test sizes: (24000, 23) (6000, 23)


In [4]:
human_map = {
    "X1": "LIMIT_BAL",
    "X2": "SEX",
    "X3": "EDUCATION",
    "X4": "MARRIAGE",
    "X5": "AGE",
    "X6": "PAY_0",
    "X7": "PAY_2",
    "X8": "PAY_3",
    "X9": "PAY_4",
    "X10": "PAY_5",
    "X11": "PAY_6",
    "X12": "BILL_AMT1",
    "X13": "BILL_AMT2",
    "X14": "BILL_AMT3",
    "X15": "BILL_AMT4",
    "X16": "BILL_AMT5",
    "X17": "BILL_AMT6",
    "X18": "PAY_AMT1",
    "X19": "PAY_AMT2",
    "X20": "PAY_AMT3",
    "X21": "PAY_AMT4",
    "X22": "PAY_AMT5",
    "X23": "PAY_AMT6"
}

df = df.rename(columns=human_map)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

feature_cols = X.columns.tolist()
print("Features:", feature_cols)

# Update train/test splits because column names changed
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
)


Features: ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

numeric_features = feature_cols

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features)]
)

# Compute class imbalance ratio
num_pos = y_train.sum()
num_neg = len(y_train) - num_pos
scale_pos_weight = num_neg / num_pos
print("scale_pos_weight =", scale_pos_weight)

xgb_clf = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=RANDOM_STATE,
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1
)

pipeline = Pipeline(steps=[
    ("preproc", preprocessor),
    ("clf", xgb_clf)
])

print("Training model…")
pipeline.fit(X_train, y_train)
print("Model trained.")


scale_pos_weight = 3.520625353173856
Training model…
Model trained.


In [6]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Predictions
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Metrics
auc = roc_auc_score(y_test, y_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)   # <-- REQUIRED for Cell 9

# Print results
print(f"AUC: {auc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion matrix:\n", cm)
print("\nClassification report:\n", report)


AUC: 0.7761
Precision: 0.4725
Recall: 0.6142
F1-score: 0.5341
Confusion matrix:
 [[3763  910]
 [ 512  815]]

Classification report:
               precision    recall  f1-score   support

           0       0.88      0.81      0.84      4673
           1       0.47      0.61      0.53      1327

    accuracy                           0.76      6000
   macro avg       0.68      0.71      0.69      6000
weighted avg       0.79      0.76      0.77      6000



In [7]:
import shap
import matplotlib.pyplot as plt

X_test_trans = pipeline.named_steps["preproc"].transform(X_test)
model = pipeline.named_steps["clf"]

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_trans)

# SHAP summary plot
plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_test_trans, feature_names=numeric_features, show=False)
plt.savefig(f"{OUTPUT_DIR}/shap_summary.png", dpi=150, bbox_inches='tight')
plt.close()

# SHAP bar plot
plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_test_trans, feature_names=numeric_features, plot_type="bar", show=False)
plt.savefig(f"{OUTPUT_DIR}/shap_bar.png", dpi=150, bbox_inches='tight')
plt.close()

print("SHAP global plots saved.")


SHAP global plots saved.


In [8]:
analysis = X_test.copy()
analysis["y_true"] = y_test.values
analysis["y_pred"] = y_pred
analysis["y_proba"] = y_proba

low_risk_idx = analysis[analysis["y_true"]==0].sort_values("y_proba").index[0]
high_risk_idx = analysis[analysis["y_true"]==1].sort_values("y_proba", ascending=False).index[0]

fn = analysis[(analysis["y_true"]==1) & (analysis["y_pred"]==0)]
fp = analysis[(analysis["y_true"]==0) & (analysis["y_pred"]==1)]

if len(fn)>0:
    surprising_idx = fn.index[0]
elif len(fp)>0:
    surprising_idx = fp.index[0]
else:
    surprising_idx = analysis.index[len(analysis)//2]

selected = {
    "low_risk": low_risk_idx,
    "high_risk": high_risk_idx,
    "surprising": surprising_idx
}

selected


{'low_risk': np.int64(1384),
 'high_risk': np.int64(16555),
 'surprising': np.int64(2157)}

In [9]:
# FINAL corrected cell: one safe loop to generate per-example SHAP outputs & explanations
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

os.makedirs(OUTPUT_DIR, exist_ok=True)
local_results = {}

# quick sanity prints
print("shap_values shape:", getattr(shap_values, "shape", None))
print("X_test shape:", X_test.shape)
print("selected:", selected)

for name, orig_idx in selected.items():
    # map original label index -> positional index for X_test / shap_values
    try:
        pos = X_test.index.get_loc(orig_idx)
    except KeyError:
        # if orig_idx is already a positional index (or X_test was reset), use it directly
        try:
            pos = int(orig_idx)
            print(f"Note: original index {orig_idx} not found; using positional {pos} as fallback.")
        except Exception as e:
            raise KeyError(f"Can't interpret selected index {orig_idx}: {e}")

    # safety check
    if pos < 0 or pos >= shap_values.shape[0]:
        raise IndexError(f"Positional index {pos} out of bounds for shap_values length {shap_values.shape[0]}")

    # fetch shap row and original features (positional)
    sv = shap_values[pos]
    orig = X_test.iloc[pos]

    # build local DataFrame (sorted by absolute SHAP)
    local_df = pd.DataFrame({
        "feature": numeric_features,
        "value": orig.values,
        "shap_value": sv
    })
    local_df["abs_shap"] = local_df["shap_value"].abs()
    local_df = local_df.sort_values("abs_shap", ascending=False).drop(columns=["abs_shap"])

    # save CSV
    csv_path = os.path.join(OUTPUT_DIR, f"local_shap_{name}.csv")
    local_df.to_csv(csv_path, index=False)

    # save horizontal bar plot of top 10 contributors (signed)
    topk = local_df.head(10).sort_values("shap_value")
    plt.figure(figsize=(8,4))
    plt.barh(topk["feature"], topk["shap_value"])
    plt.xlabel("SHAP value")
    plt.title(f"Top SHAP contributors ({name})")
    plt.tight_layout()
    plot_path = os.path.join(OUTPUT_DIR, f"local_shap_bar_{name}.png")
    plt.savefig(plot_path, dpi=150)
    plt.close()

    # prepare textual explanation using the 'analysis' DataFrame when possible
    if orig_idx in analysis.index:
        pred_proba = float(analysis.loc[orig_idx, "y_proba"])
        true_label = int(analysis.loc[orig_idx, "y_true"])
    else:
        pred_proba = float(analysis.iloc[pos]["y_proba"])
        true_label = int(analysis.iloc[pos]["y_true"])

    top_pos = local_df[local_df["shap_value"] > 0].head(5)
    top_neg = local_df[local_df["shap_value"] < 0].head(5)

    lines = []
    lines.append(f"--- {name.upper()} CASE (orig index {orig_idx}, pos {pos}) ---")
    lines.append(f"True label: {true_label}, Predicted probability: {pred_proba:.4f}\n")
    lines.append("Top 5 risk-increasing features:")
    for _, r in top_pos.iterrows():
        lines.append(f"  + {r['feature']} = {r['value']}, SHAP={r['shap_value']:.4f}")
    lines.append("\nTop 5 risk-decreasing features:")
    for _, r in top_neg.iterrows():
        lines.append(f"  - {r['feature']} = {r['value']}, SHAP={r['shap_value']:.4f}")

    explanation_text = "\n".join(lines)
    txt_path = os.path.join(OUTPUT_DIR, f"explanation_{name}.txt")
    with open(txt_path, "w") as f:
        f.write(explanation_text)

    # store and print
    local_results[name] = {"pos": pos, "csv": csv_path, "plot": plot_path, "txt": txt_path, "df": local_df}
    print(f"\n{name.upper()} EXPLANATION (orig index {orig_idx}, pos {pos}):")
    print(local_df.head(10))

print("\nAll per-example outputs saved to:", OUTPUT_DIR)


shap_values shape: (6000, 23)
X_test shape: (6000, 23)
selected: {'low_risk': np.int64(1384), 'high_risk': np.int64(16555), 'surprising': np.int64(2157)}

LOW_RISK EXPLANATION (orig index 1384, pos 5368):
      feature   value  shap_value
18   PAY_AMT2  120927   -0.945953
17   PAY_AMT1   83754   -0.736917
5       PAY_0      -1   -0.424108
0   LIMIT_BAL  300000   -0.357485
19   PAY_AMT3       0   -0.248942
11  BILL_AMT1   42189   -0.165250
15  BILL_AMT5   82475   -0.163355
4         AGE      27   -0.162768
22   PAY_AMT6   92250   -0.155413
21   PAY_AMT5   25213   -0.090043

HIGH_RISK EXPLANATION (orig index 16555, pos 469):
      feature  value  shap_value
5       PAY_0      3    1.385520
10      PAY_6      7    0.423067
0   LIMIT_BAL  10000    0.329796
14  BILL_AMT4   2300    0.249434
6       PAY_2      2    0.224244
11  BILL_AMT1   2300    0.193294
19   PAY_AMT3      0    0.184054
9       PAY_5      7    0.183551
8       PAY_4      7    0.162810
7       PAY_3      2    0.162629

SURPR

In [11]:
# Quick cell to (re)compute feat_imp and save top5 + executive summary
import os
from datetime import datetime
import numpy as np
import pandas as pd

# ensure OUTPUT_DIR exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- compute feat_imp from shap_values if missing ---
# shap_values: ndarray shape (n_test, n_features)
# numeric_features: list of feature names (length n_features)
try:
    feat_imp  # if exists, do nothing
    print("feat_imp already defined.")
except NameError:
    print("Computing feat_imp from shap_values...")
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    feat_imp = pd.DataFrame({
        "feature": numeric_features,
        "mean_abs_shap": mean_abs_shap
    }).sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)
    print("Computed feat_imp.")

# Save top-5 global SHAP
top5 = feat_imp.head(5)
top5_path = os.path.join(OUTPUT_DIR, "top5_shap.csv")
top5.to_csv(top5_path, index=False)
print("Saved top-5 SHAP ->", top5_path)
print("Top-5 features:\n", top5)

# If metrics (auc, precision, recall, f1, cm, report) are missing, warn
missing = [v for v in ["auc","precision","recall","f1","cm","report","X_train","X_test"] if v not in globals()]
if missing:
    print("Warning: the following variables are not in session and are required for the executive summary:", missing)
    print("If these are missing, re-run the model evaluation cell (Cell 5) before proceeding.")
else:
    # Executive summary (auto-generate)
    exec_path = os.path.join(OUTPUT_DIR, "executive_summary.txt")
    exec_text = f"""Executive summary — Default of Credit Card Clients (auto-generated)
Generated: {datetime.now().isoformat()}

Model: XGBoost classifier
Train / Test sizes: {X_train.shape[0]} / {X_test.shape[0]}

Key metrics (test):
- AUC: {auc:.4f}
- Precision: {precision:.4f}
- Recall: {recall:.4f}
- F1: {f1:.4f}

Top 5 global SHAP drivers:
{chr(10).join([f'{i+1}. {row.feature} (mean|SHAP|={row.mean_abs_shap:.6f})' for i,row in top5.reset_index(drop=True).iterrows()])}

Business recommendations:
1) Use the top SHAP drivers in underwriting rules and manual review workflows.
2) For surprising cases flagged by the model, require additional manual checks.
3) Monitor feature drift and SHAP distribution monthly; retrain if performance drops.
4) Consider threshold adjustment for operational balance between recall/precision.

Artifacts saved in: {OUTPUT_DIR}
"""
    with open(exec_path, "w") as f:
        f.write(exec_text)
    print("Saved executive summary ->", exec_path)

    # Print short summary for quick copy/paste
    print("\n=== Quick summary ===")
    print(f"AUC: {auc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")
    print("\n--- Top 5 SHAP ---")
    print(top5.to_string(index=False))


Computing feat_imp from shap_values...
Computed feat_imp.
Saved top-5 SHAP -> /content/drive/MyDrive/project_outputs/top5_shap.csv
Top-5 features:
      feature  mean_abs_shap
0      PAY_0       0.554533
1  LIMIT_BAL       0.224891
2  BILL_AMT1       0.158520
3   PAY_AMT2       0.127405
4   PAY_AMT1       0.114219
Saved executive summary -> /content/drive/MyDrive/project_outputs/executive_summary.txt

=== Quick summary ===
AUC: 0.7761
Precision: 0.4725
Recall: 0.6142
F1: 0.5341

--- Top 5 SHAP ---
  feature  mean_abs_shap
    PAY_0       0.554533
LIMIT_BAL       0.224891
BILL_AMT1       0.158520
 PAY_AMT2       0.127405
 PAY_AMT1       0.114219


In [12]:
# CELL 9 — Save final metrics, top-5 SHAP, and executive summary
import os
from datetime import datetime
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Metrics (assumes auc, precision, recall, f1, cm, report variables exist)
metrics_path = os.path.join(OUTPUT_DIR, "metrics.txt")
with open(metrics_path, "w") as f:
    f.write("Model evaluation metrics (test set)\n")
    f.write("===============================\n")
    f.write(f"AUC: {auc:.4f}\n")
    f.write(f"Precision: {precision:.4f}\n")
    f.write(f"Recall: {recall:.4f}\n")
    f.write(f"F1: {f1:.4f}\n\n")
    f.write("Confusion matrix:\n")
    f.write(np.array2string(cm))
    f.write("\n\nClassification report:\n")
    f.write(report)
print("Saved metrics ->", metrics_path)

# Top-5 global SHAP (feat_imp DataFrame assumed to exist)
top5 = feat_imp.head(5)
top5_path = os.path.join(OUTPUT_DIR, "top5_shap.csv")
top5.to_csv(top5_path, index=False)
print("Saved top-5 SHAP ->", top5_path)
print("Top-5 features:\n", top5)

# Executive summary (auto-generate, customize as needed)
exec_path = os.path.join(OUTPUT_DIR, "executive_summary.txt")
exec_text = f"""Executive summary — Default of Credit Card Clients (auto-generated)
Generated: {datetime.now().isoformat()}

Model: XGBoost classifier
Train / Test sizes: {X_train.shape[0]} / {X_test.shape[0]}

Key metrics (test):
- AUC: {auc:.4f}
- Precision: {precision:.4f}
- Recall: {recall:.4f}
- F1: {f1:.4f}

Top 5 global SHAP drivers:
{chr(10).join([f'{i+1}. {row.feature} (mean|SHAP|={row.mean_abs_shap:.6f})' for i,row in top5.reset_index(drop=True).iterrows()])}

Business recommendations:
1) Use the top SHAP drivers in underwriting rules and manual review workflows.
2) For surprising cases flagged by the model, require additional manual checks.
3) Monitor feature drift and SHAP distribution monthly; retrain if performance drops.
4) Consider threshold adjustment for operational balance between recall/precision.

Artifacts saved in: {OUTPUT_DIR}
"""
with open(exec_path, "w") as f:
    f.write(exec_text)
print("Saved executive summary ->", exec_path)

# Print short summary for quick copy/paste
print("\n=== Quick summary ===")
print(open(metrics_path).read())
print("--- Top 5 SHAP ---")
print(top5.to_string(index=False))


Saved metrics -> /content/drive/MyDrive/project_outputs/metrics.txt
Saved top-5 SHAP -> /content/drive/MyDrive/project_outputs/top5_shap.csv
Top-5 features:
      feature  mean_abs_shap
0      PAY_0       0.554533
1  LIMIT_BAL       0.224891
2  BILL_AMT1       0.158520
3   PAY_AMT2       0.127405
4   PAY_AMT1       0.114219
Saved executive summary -> /content/drive/MyDrive/project_outputs/executive_summary.txt

=== Quick summary ===
Model evaluation metrics (test set)
AUC: 0.7761
Precision: 0.4725
Recall: 0.6142
F1: 0.5341

Confusion matrix:
[[3763  910]
 [ 512  815]]

Classification report:
              precision    recall  f1-score   support

           0       0.88      0.81      0.84      4673
           1       0.47      0.61      0.53      1327

    accuracy                           0.76      6000
   macro avg       0.68      0.71      0.69      6000
weighted avg       0.79      0.76      0.77      6000

--- Top 5 SHAP ---
  feature  mean_abs_shap
    PAY_0       0.554533
LIMIT